**Objetivo:** Leer y estandarizar los datasets (SIES, DEMRE, IVM, georreferenciación, etc.)

- Cargar datos de todas las fuentes
- Verificar formatos y columnas clave
- Establecer claves de unión (`MRUN`, `RBD`, códigos de institución)
- Guardar versiones limpias en `data/processed/` (opcional)

In [ ]:
import utils
from pathlib import Path

In [ ]:
# Lectura 
path = "../data/raw/2021"
list( Path(path).glob('*'))

In [ ]:
# Ruta base
path = '../data/raw'
path_base = Path(path)
year = 2021

# Obtener paths del año
path_year = utils.obtener_paths_anio(path_base, year)
path_year

In [ ]:
# Leer conjuntos individuales
set_a = utils.leer_conjunto_a_desde_path(path_year['matricula'], utils.region_dict)
set_b = utils.leer_conjunto_b_desde_path(path_year['puntajes'], utils.region_dict)
set_c = utils.leer_conjunto_c_desde_path(path_year['establecimientos'])
set_d = utils.leer_conjunto_d_desde_path(path_year['inmuebles_ies'], utils.region_dict)
set_e = utils.leer_conjunto_e_desde_path(path_year['ivm'])

In [ ]:
# Filtrar A1
resultados_a = utils.generar_conjuntos_filtrados(set_a, year)
set_a0 = resultados_a['A0']
set_aa = resultados_a['A']
set_a1 = resultados_a['A1']


In [ ]:
# Carpeta de salida
output_path = Path(f'../data/clean/{year}')
output_path.mkdir(parents=True, exist_ok=True)

# Guardar conjuntos individuales
set_a.to_csv(output_path / 'set_a.csv', index=False)
set_a0.to_csv(output_path / 'set_a0.csv', index=False)
set_aa.to_csv(output_path / 'set_aa.csv', index=False)
set_a1.to_csv(output_path / 'set_a1.csv', index=False)
set_b.to_csv(output_path / 'set_b.csv', index=False)
set_c.to_csv(output_path / 'set_c.csv', index=False)
set_d.to_csv(output_path / 'set_d.csv', index=False)
set_e.to_csv(output_path / 'set_e.csv', index=False)

In [ ]:
# Generar y guardar conjuntos integrados
utils.generar_conjuntos_abcde(
    set_a=set_a1,
    set_b=set_b,
    set_c=set_c,
    set_d=set_d,
    set_e=set_e,
    output_path=output_path
)

**Objetivo:** Explorar y analizar los datos limpios generados en la etapa de lectura.

- Revisar estructura general y dimensiones de los datasets  
- Analizar variables clave y su distribución  
- Identificar outliers, datos faltantes y relaciones relevantes  
- Generar visualizaciones exploratorias para apoyar el análisis  


In [12]:
import plots
import pandas as pd
from pathlib import Path

In [24]:
# usar la función
orden_regiones = [
    'Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo',
    'Valparaíso', 'Metropolitana', "Lib. Gral B. O'Higgins", 'Maule', 'Ñuble',
    'Biobío', 'La Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén', 'Magallanes'
]

tipos_universidad = [
    'Universidades Estatales CRUCH', 
    'Universidades Privadas', 
    'Universidades Privadas CRUCH'
]


In [25]:
import plotly.graph_objects as go
import pandas as pd

def plot_tipo_universidad_por_region_plotly(df, tipos_universidad, orden_regiones):
    """
    Gráfico de barras apiladas horizontales en Plotly mostrando el porcentaje de tipos de universidad por región.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con las columnas 'cod_inst', 'region_sede' y 'tipo_inst_3'.
    tipos_universidad : list
        Lista con los tipos de universidad a filtrar.
    orden_regiones : list
        Lista con las regiones ordenadas de norte a sur.
    """

    # Filtrar datos y eliminar duplicados
    df_filtered = df[df['tipo_inst_3'].isin(tipos_universidad)].copy()
    df_filtered_unique = df_filtered.drop_duplicates(subset=['cod_inst', 'region_sede', 'tipo_inst_3'])

    # Ordenar regiones como categoría
    df_filtered_unique['region_sede'] = pd.Categorical(
        df_filtered_unique['region_sede'], 
        categories=orden_regiones, 
        #ordered=True
    )

    # Agrupar y calcular porcentaje
    df_grouped = df_filtered_unique.groupby(['region_sede', 'tipo_inst_3'], observed=True).size().unstack(fill_value=0)
    df_grouped = df_grouped.div(df_grouped.sum(axis=1), axis=0) * 100
    df_grouped = df_grouped.reindex(orden_regiones)

    # Colores (puedes personalizarlos)
    colores = {
        'Universidades Estatales CRUCH': 'blue',
        'Universidades Privadas': 'green',
        'Universidades Privadas CRUCH': 'orange',
    }

    # Crear gráfico Plotly
    fig = go.Figure()

    for tipo in df_grouped.columns:
        fig.add_trace(go.Bar(
            y=df_grouped.index,
            x=df_grouped[tipo],
            name=tipo,
            orientation='h',
            marker=dict(color=colores.get(tipo, None)),
            text=df_grouped[tipo].apply(lambda x: f'{x:.0f}%'),
            textposition='inside',
            insidetextanchor='middle',
        ))

    # Layout
    fig.update_layout(
        barmode='stack',
        title='Porcentaje Tipo Universidad por Región',
        xaxis_title='Porcentaje',
        yaxis_title='Región',
        legend_title='Tipo de Universidad',
        height=600,
        margin=dict(l=50, r=200, t=50, b=50),
    )

    fig.show()


In [28]:
import plotly.graph_objects as go
import pandas as pd

def plot_tipodepen_por_region_plotly(df, regiones_dict, tipodepen_dict, index_order):
    """
    Genera un gráfico de barras apiladas con el porcentaje de TIPO_DEPEN por región (versión Plotly).

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con columnas: 'NOM_RBD', 'COD_REG_RB' y 'TIPO_DEPEN'.
    regiones_dict : dict
        Diccionario para mapear ID de región a nombre.
    tipodepen_dict : dict
        Diccionario para mapear número de TIPO_DEPEN a nombre.
    index_order : list
        Lista con las regiones ordenadas de norte a sur.
    """

    # Filtrar y eliminar duplicados
    df_filtered = df.copy()
    df_filtered_unique = df_filtered.drop_duplicates(subset=['NOM_RBD', 'COD_REG_RB', 'TIPO_DEPEN'])

    # Agrupar y calcular porcentaje
    df_grouped = df_filtered_unique.groupby(['COD_REG_RB', 'TIPO_DEPEN'], observed=True).size().unstack(fill_value=0)
    df_grouped = df_grouped.div(df_grouped.sum(axis=1), axis=0) * 100

    # Mapear nombres
    df_grouped.index = df_grouped.index.map(regiones_dict)
    df_grouped.columns = df_grouped.columns.map(tipodepen_dict)

    # Eliminar duplicados en index_order y reordenar (de norte a sur)
    index_order = sorted(set(index_order), key=index_order.index)
    df_grouped = df_grouped.reindex(index_order[::-1])

    # Colores personalizados
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

    # Crear gráfico Plotly
    fig = go.Figure()

    for i, tipo in enumerate(df_grouped.columns):
        fig.add_trace(go.Bar(
            y=df_grouped.index,
            x=df_grouped[tipo],
            name=tipo,
            orientation='h',
            marker=dict(color=colors[i % len(colors)]),
            text=df_grouped[tipo].apply(lambda x: f'{x:.0f}%' if x > 0.99 else ''),
            textposition='inside',
            insidetextanchor='middle',
            hovertemplate=f'{tipo}<br>%{{y}}: %{{x:.1f}}%<extra></extra>'
        ))

    # Layout personalizado
    fig.update_layout(
        barmode='stack',
        title='Porcentaje de TIPO_DEPEN para cada región',
        xaxis_title='Porcentaje (%)',
        yaxis_title='Región',
        legend_title='TIPO_DEPEN',
        height=700,
        margin=dict(l=50, r=200, t=50, b=50),
    )

    fig.show()


In [26]:
# Lectura de datos

year = 2021
DATA_DIR = Path(f"../data/clean/{year}")

# Buscar archivos
archivos = list(DATA_DIR.glob('set_*.csv'))

# Leer archivos y guardar en un diccionario
data = {
    archivo.stem: pd.read_csv(archivo)
    for archivo in archivos
}

In [27]:
plot_tipo_universidad_por_region_plotly(data['set_a'], tipos_universidad, orden_regiones)

C:\Users\franc\AppData\Local\Temp\ipykernel_28004\2650108794.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_unique['region_sede'] = pd.Categorical(


In [29]:
# Diccionario ID Región -> Nombre Región
regiones_dict = {
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    6: 'Región Metropolitana de Santiago',
    7: 'Región del Libertador Gral. Bernardo O’Higgins',
    8: 'Región del Maule',
    9: 'Región de Biobío',
    10: 'Región de La Araucanía',
    11: 'Región de Los Ríos',
    12: 'Región de Los Lagos',
    13: 'Región de Aysén del Gral. Carlos Ibáñez del Campo',
    14: 'Región de Magallanes y de la Antártica Chilena',
    15: 'Región de Arica y Parinacota',
    16: 'Región de Ñuble'
}

# Diccionario TIPO_DEPEN -> Nombre
tipodepen_dict = {
    1: 'Municipal',
    2: 'Particular Subvencionado',
    3: 'Particular Pagado',
    4: 'Corp. Administración Delegada',
    5: 'Servicio Local de Educación'
}

# Orden de las regiones de norte a sur
index_order = [
    'Región de Arica y Parinacota',
    'Región de Tarapacá',
    'Región de Antofagasta',
    'Región de Atacama',
    'Región de Coquimbo',
    'Región de Valparaíso',
    'Región Metropolitana de Santiago',
    'Región del Libertador Gral. Bernardo O’Higgins',
    'Región del Maule',
    'Región de Ñuble',
    'Región de Biobío',
    'Región de La Araucanía',
    'Región de Los Ríos',
    'Región de Los Lagos',
    'Región de Aysén del Gral. Carlos Ibáñez del Campo',
    'Región de Magallanes y de la Antártica Chilena'
]


plot_tipodepen_por_region_plotly(
    df=data['set_c'],
    regiones_dict=regiones_dict,
    tipodepen_dict=tipodepen_dict,
    index_order=index_order
)